In [1]:
import org.apache.spark.mllib.tree.DecisionTree
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.classification.{SVMModel, SVMWithSGD}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.util.MLUtils

In [2]:
val data = MLUtils.loadLibSVMFile(sc, "data/01_credit_data.txt")
val splits = data.randomSplit(Array(0.7, 0.3))
val (trainingData, testData) = (splits(0), splits(1))

data = MapPartitionsRDD[6] at map at MLUtils.scala:86
splits = Array(MapPartitionsRDD[7] at randomSplit at <console>:36, MapPartitionsRDD[8] at randomSplit at <console>:36)
trainingData = MapPartitionsRDD[7] at randomSplit at <console>:36
testData = MapPartitionsRDD[8] at randomSplit at <console>:36


MapPartitionsRDD[8] at randomSplit at <console>:36

* Dividimos la data total entre 60% y 40% para entrenamiento y validación.

In [3]:
val splits = data.randomSplit(Array(0.6, 0.4), seed = 11L)
val training = splits(0).cache()
val test = splits(1)

splits = Array(MapPartitionsRDD[9] at randomSplit at <console>:36, MapPartitionsRDD[10] at randomSplit at <console>:36)
training = MapPartitionsRDD[9] at randomSplit at <console>:36
test = MapPartitionsRDD[10] at randomSplit at <console>:36


MapPartitionsRDD[10] at randomSplit at <console>:36

* Entrenamos con el algoritmo SVM.

In [4]:
val numIterations = 100
val model = SVMWithSGD.train(training, numIterations)

numIterations = 100
model = org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 8, numClasses = 2, threshold = 0.0


org.apache.spark.mllib.classification.SVMModel: intercept = 0.0, numFeatures = 8, numClasses = 2, threshold = 0.0

* Se ejecuta la validación.

In [5]:
val scoreAndLabels = test.map { point =>
  val score = model.predict(point.features)
  (score, point.label)
}

scoreAndLabels = MapPartitionsRDD[214] at map at <console>:37


MapPartitionsRDD[214] at map at <console>:37

* Evaluacion el error del modelo generado

In [6]:
val metrics = new BinaryClassificationMetrics(scoreAndLabels)
val auROC = metrics.areaUnderROC()

metrics = org.apache.spark.mllib.evaluation.BinaryClassificationMetrics@5c58cc81
auROC = 0.8008245394387097


0.8008245394387097

In [7]:
println(s"Area under ROC = $auROC")

Area under ROC = 0.8008245394387097


In [8]:
model.toPMML("models/01_credit_SVM_model_60_40.pmml")